In [1]:
!source /home/nikhil/Desktop/repo/KamleshThesis/env/bin/activate

In [ ]:
# !git clone https://github.com/gnikhil27/deepface.git
# !mv deepface df
# !mv df/* .

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import os

import keras.layers as layers
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from tensorflow.keras import Model

from deepface.basemodels.ArcFace import loadModel
from deepface.commons import functions, distance
import matplotlib.pyplot as plt
import cv2


2022-11-12 19:51:56.829367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 19:51:57.465181: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 19:51:59.512624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-12 19:51:59.512735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [4]:
model_mid = loadModel(nname='Test')
model_mid.summary()

2022-11-12 19:52:17.703403: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-12 19:52:17.703487: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (laptop): /proc/driver/nvidia/version does not exist
2022-11-12 19:52:17.703911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 19:52:22.504560: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2022-11-12 19:52:22.841401: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2022-11-12 19:52:22

arcface_weights.h5  will be downloaded to  arcface_weights.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /home/nikhil/Desktop/repo/deepface/arcface_weights.h5
100%|██████████| 137M/137M [00:18<00:00, 7.36MB/s] 


Model: "Test_ArcFace"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 Testconv1_pad (ZeroPadding2D)  (None, 114, 114, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Testconv1_conv (Conv2D)        (None, 112, 112, 64  1728        ['Testconv1_pad[0][0]']          
                                )                                                                 
                                                                                       

2022-11-12 19:52:44.201244: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.


 rmalization)                                                                                     
                                                                                                  
 Testconv3_block1_1_prelu (PReL  (None, 56, 56, 128)  128        ['Testconv3_block1_2_bn[0][0]']  
 U)                                                                                               
                                                                                                  
 Testconv3_block1_2_pad (ZeroPa  (None, 58, 58, 128)  0          ['Testconv3_block1_1_prelu[0][0]'
 dding2D)                                                        ]                                
                                                                                                  
 Testconv3_block1_0_conv (Conv2  (None, 28, 28, 128)  8192       ['Testconv2_block3_add[0][0]']   
 D)                                                                                               
          

In [5]:
# Define the Contrastive Loss
def contrastive_loss(y, preds, margin=1):
    y = tf.cast(y, preds.dtype)
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)

    return loss

In [6]:
# Distance Layer
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance
    between the embeddings
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, compare):
        # return distance.findCosineDistance(anchor, compare)
        sum_squared = K.sum(K.square(anchor - compare), axis=1, keepdims=True)
        return K.sqrt(K.maximum(sum_squared, K.epsilon()))

In [ ]:
#decide input shape
input_shape_x, input_shape_y = functions.find_input_shape(model_mid)
# t_shape = (input_shape_y, input_shape_x, 3) 

# anchor_input = layers.Input(name="anchor", shape=t_shape)
# compare_input = layers.Input(name="compare", shape=t_shape)

anchor_input = loadModel(nname='anchor')
compare_input = loadModel(nname='compare')


distances = DistanceLayer()(
    anchor_input.output,
    compare_input.output,
)

outputs = layers.Dense(1, activation = "sigmoid") (distances)

model = Model(
    inputs=[anchor_input.input, compare_input.input], outputs=outputs,
    name = "ArcFace_Training"
)

In [ ]:
model.compile(loss="mean_absolute_percentage_error", optimizer="adam", metrics=["accuracy"])
model.summary()

## Load DataSet

In [ ]:
def get_img(img_path, input_shape_y = input_shape_y, input_shape_x = input_shape_x, enforce_detection = True
		, detector_backend = 'opencv', align = True, normalization = 'base'):
	'''Returns Image when given the following details'''
	#detect and align
	img = functions.preprocess_face(img = img_path
		, target_size=(input_shape_y, input_shape_x)
		, enforce_detection = enforce_detection
		, detector_backend = detector_backend
		, align = align)

	#custom normalization
	img = functions.normalize_input(img = img, normalization = normalization)
	return img

In [ ]:
# Get the data directory paths
# data_folder = "/home/nikhil/Desktop/repo/KamleshThesis/data/LFW/lfw/"
data_folder = "/content/drive/MyDrive/KamleshThesis/lfw/lfw/"

img_type = ['jpg']

In [ ]:
name_list = os.listdir(data_folder)
name_list = name_list[:100]
face_labels = []
face_images = []

print("Processing ", len(name_list), " Folders\n\n")
for fold in name_list:
    per_path = os.path.join(data_folder, fold)
    if os.path.isdir(per_path):
        for img in os.listdir(per_path):
            img_path = os.path.join(per_path, img)
            if os.path.isfile(img_path) and img[-3:].lower() in img_type:
                try:
                    img_arr = get_img(img_path)
                    print(img_path)
                    face_labels.append(name_list.index(fold))
                    face_images.append(img_arr)
                except:
                    print('Cannot load '+fold)


In [ ]:
img_path = "/content/drive/MyDrive/KamleshThesis/lfw/lfw/Ruth_Bader_Ginsburg/Ruth_Bader_Ginsburg_0001.jpg"
# img = cv2.imread(img_path, cv2.IMREAD_ANYCOLOR)
img = get_img(img_path)

def show_img(img):
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.title('my picture')
  plt.show()

show_img(img[0])
img.shape

In [ ]:
img_len = len(face_images)
if img_len == 1:
    print("removing Extra")
    face_images = face_images[0]

if len(face_images[0]) != input_shape_x:
    for index, face in enumerate(face_images):
        face_images[index] = face[0]

if img_len == len(face_labels):
    print('success')
else:
    print('Failiure')

In [ ]:
print(len(face_images[0][0]))


In [ ]:
# Function to generate image pairs
def generate_image_pairs(images, labels):
    # Generate index for each label
    unique_labels = np.unique(labels)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels) if
                                       label == curr_label])
    
    # Generate image pairs and labels
    pair_images = []
    pair_labels = []
    for index, image in enumerate(images):
        pos_indices = label_wise_indices.get(labels[index])
        pos_image = images[np.random.choice(pos_indices)]
        pair_images.append((image, pos_image))
        pair_labels.append(1)

        neg_indices = np.where(labels != labels[index])
        neg_image = images[np.random.choice(neg_indices[0])]
        pair_images.append((image, neg_image))
        pair_labels.append(0)
        
    return np.array(pair_images), np.array(pair_labels)

In [ ]:
images_dataset, labels_dataset = generate_image_pairs(face_images, face_labels)
images_dataset, labels_dataset = shuffle(images_dataset, labels_dataset)

In [ ]:
show_img(images_dataset[:, 0, :][1])

In [ ]:
images_dataset.shape

In [ ]:
labels_dataset

In [ ]:
history = model.fit(
    [images_dataset[:, 0, :], images_dataset[:, 1, :]], 
    labels_dataset,
    epochs=50, 
    validation_split = 0.2, 
    batch_size = 64, 
    # callbacks=[
    #     ModelCheckpoint(
    #         'model.hdf5',
    #         verbose=1, 
    #         save_best_only=True
    #     )
    # ]
)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.legend(["Loss", "Validation Loss", "Accuracy", "Validation Accuracy"])

In [ ]:
def test_visualize(images, n = 5):
    """ Visualize a few images """

    def show(ax, image):
        ax.imshow(image)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    fig = plt.figure(figsize=(9, 9)) 
    axs = fig.subplots(1, n)
    for i in range(n):
        show(axs[i], images[i])


In [ ]:
import random


In [ ]:
test_indices = [0, 1, 2, 3, 4]
test_index = random.sample(range(0, 50), 1)[0]
test_image = face_images[test_index]

compare_images = []
for i in range(5):
    index = random.sample(range(i * 10, (i + 1) * 10), 1)[0]
    image = face_images[index]
    compare_images.append(image)
plt.imshow(test_image)